In [10]:
import pandas as pd
import numpy as np
import sqlite3
import os

# Caminho da pasta com os dados
dados = "../data/raw"

# Arquivos CSV
pedidos = dados + "/olist_orders_dataset.csv"
produtos = dados + "/olist_products_dataset.csv"
vendedores = dados + "/olist_sellers_dataset.csv"
itens = dados + "/olist_order_items_dataset.csv"
reviews = dados + "/olist_order_reviews_dataset.csv"
geoloc = dados + "/olist_geolocation_dataset.csv"
clientes = dados + "/olist_customers_dataset.csv"
pagamentos = dados + "/olist_order_payments_dataset.csv"
categoria_nome = dados + "/product_category_name_translation.csv"

# Carregar arquivos CSV
df_pedidos = pd.read_csv(pedidos)
df_produtos = pd.read_csv(produtos)
df_vendedores = pd.read_csv(vendedores)
df_itens = pd.read_csv(itens)
df_reviews = pd.read_csv(reviews)
df_geoloc = pd.read_csv(geoloc)
df_clientes = pd.read_csv(clientes)
df_pagamentos = pd.read_csv(pagamentos)
df_categoria_nome = pd.read_csv(categoria_nome)

In [11]:
# Função para ver nulos e duplicatas
def check_df(df, name):
    print(f"{name}:")
    print("Nulls:\n", df.isnull().sum())
    print("Duplicatas:", df.duplicated().sum())
    print("-" * 40)

# Checar todos os dataframes
dfs = {
    "Pedidos": df_pedidos,
    "Produtos": df_produtos,
    "Vendedores": df_vendedores,
    "Itens": df_itens,
    "Reviews": df_reviews,
    "Geolocalização": df_geoloc,
    "Clientes": df_clientes,
    "Pagamentos": df_pagamentos,
    "Categorias": df_categoria_nome,
}

for name, df in dfs.items():
    check_df(df, name)

# Remover duplicatas
for df in dfs.values():
    df.drop_duplicates(inplace=True)

Pedidos:
Nulls:
 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64
Duplicatas: 0
----------------------------------------
Produtos:
Nulls:
 product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64
Duplicatas: 0
----------------------------------------
Vendedores:
Nulls:
 seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64
Duplicatas: 0
----------------------------------------
Itens:
Nulls:
 orde

In [12]:
# Datas do df_pedidos
colunas_datas = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in colunas_datas:
    df_pedidos[col] = pd.to_datetime(df_pedidos[col])

In [13]:
df_produtos = df_produtos.merge(
    df_categoria_nome, on='product_category_name', how='left'
)

In [14]:
# Criar banco em memória ou arquivo
conn = sqlite3.connect("../data/processed/ecommerce_olist.db")  # altere se quiser manter em memória: ":memory:"

# Salvar os DataFrames como tabelas no SQLite
df_pedidos.to_sql("pedidos", conn, index=False, if_exists="replace")
df_produtos.to_sql("produtos", conn, index=False, if_exists="replace")
df_vendedores.to_sql("vendedores", conn, index=False, if_exists="replace")
df_itens.to_sql("itens", conn, index=False, if_exists="replace")
df_reviews.to_sql("reviews", conn, index=False, if_exists="replace")
df_geoloc.to_sql("geoloc", conn, index=False, if_exists="replace")
df_clientes.to_sql("clientes", conn, index=False, if_exists="replace")
df_pagamentos.to_sql("pagamentos", conn, index=False, if_exists="replace")
df_categoria_nome.to_sql("categorias", conn, index=False, if_exists="replace")

71

In [15]:
# Criar diretório se não existir
os.makedirs("../data/processed", exist_ok=True)

# Salvar arquivos limpos
df_pedidos.to_csv("../data/processed/pedidos.csv", index=False)
df_produtos.to_csv("../data/processed/produtos.csv", index=False)
df_itens.to_csv("../data/processed/itens.csv", index=False)
df_vendedores.to_csv("../data/processed/vendedores.csv", index=False)
df_reviews.to_csv("../data/processed/reviews.csv", index=False)
df_geoloc.to_csv("../data/processed/geoloc.csv", index=False)
df_clientes.to_csv("../data/processed/clientes.csv", index=False)
df_pagamentos.to_csv("../data/processed/pagamentos.csv", index=False)
df_categoria_nome.to_csv("../data/processed/categorias.csv", index=False)


In [16]:
from geopy.distance import geodesic

# Coordenadas do vendedor em Brasília
lat_vendedor, lon_vendedor = -15.7801, -47.9292

# Eliminar nulos e manter apenas um registro por CEP
df_geoloc_filtrado = df_geoloc.dropna().drop_duplicates('geolocation_zip_code_prefix')

# Calcular distância
df_distancias = df_geoloc_filtrado.copy()
df_distancias['distance_km'] = df_distancias.apply(
    lambda row: geodesic((lat_vendedor, lon_vendedor), (row['geolocation_lat'], row['geolocation_lng'])).km,
    axis=1
)

# Salvar no disco
df_distancias[['geolocation_zip_code_prefix', 'distance_km']].to_csv("../data/processed/distancias.csv", index=False)

In [17]:
# Carregar distâncias já calculadas
df_distancias = pd.read_csv("../data/processed/distancias.csv")

# Abrir conexão
conn = sqlite3.connect("../data/processed/ecommerce_olist.db")

# Salvar no banco
df_distancias.to_sql("distancias", conn, index=False, if_exists="replace")

# Fechar conexão
conn.close()